In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random

# Setup
faker = Faker()
np.random.seed(42)
random.seed(42)

n = 1000
today = datetime.today()
data = []

# Prezzi abbonamento coerenti
prezzi = {
    'mensile': 30,
    'trimestrale': 80,
    'annuale': 280
}

# Profili comportamentali
def generate_customer(i):
    profile_type = random.choices(
        ['fedele', 'incostante', 'a_rischio'],
        weights=[0.5, 0.3, 0.2]
    )[0]

    sesso = random.choice(['M', 'F'])
    età = np.random.randint(18, 65)
    data_iscrizione = faker.date_between(start_date='-2y', end_date='-60d')
    tipo_abbonamento = random.choices(
        ['mensile', 'trimestrale', 'annuale'],
        weights=[0.5, 0.3, 0.2]
    )[0]
    prezzo_abbonamento = prezzi[tipo_abbonamento]

    if profile_type == 'fedele':
        giorni_da_ultima_presenza = np.random.randint(0, 20)
        media_presenze_sett = round(np.random.normal(2.5, 0.5), 2)
        ha_rinnovato = True
        churn = 0

    elif profile_type == 'incostante':
        giorni_da_ultima_presenza = np.random.randint(15, 60)
        media_presenze_sett = round(np.random.normal(1.5, 0.7), 2)
        ha_rinnovato = random.choices([True, False], weights=[0.6, 0.4])[0]
        churn = int(not ha_rinnovato and giorni_da_ultima_presenza > 30)

    else:  # a_rischio
        giorni_da_ultima_presenza = np.random.randint(45, 120)
        media_presenze_sett = round(np.random.normal(0.5, 0.3), 2)
        ha_rinnovato = False
        churn = 1

    ultima_presenza = today - timedelta(days=giorni_da_ultima_presenza)

    return {
        'customer_id': f'CUST{i:04d}',
        'sesso': sesso,
        'età': età,
        'data_iscrizione': data_iscrizione,
        'tipo_abbonamento': tipo_abbonamento,
        'prezzo_abbonamento': prezzo_abbonamento,
        'ultima_presenza': ultima_presenza.date(),
        'media_presenze_sett': max(media_presenze_sett, 0),
        'giorni_da_ultima_presenza': giorni_da_ultima_presenza,
        'ha_rinnovato': ha_rinnovato,
        'churn': churn
    }

# Genera i dati
for i in range(n):
    data.append(generate_customer(i))

# Crea DataFrame
df = pd.DataFrame(data)

# Salva su file
file_path = "dataset1.csv"
df.to_csv(file_path, index=False)

df.head()


,customer_id,sesso,età,data_iscrizione,tipo_abbonamento,prezzo_abbonamento,ultima_presenza,media_presenze_sett,giorni_da_ultima_presenza,ha_rinnovato,churn
0,CUST0000,M,56,2023-07-29,trimestrale,80,2025-05-10,1.88,43,True,0
1,CUST0001,M,38,2025-01-06,trimestrale,80,2025-06-16,2.19,6,True,0
2,CUST0002,M,36,2024-05-28,trimestrale,80,2025-02-23,0.00,119,False,1
3,CUST0003,M,57,2023-08-06,mensile,30,2025-06-20,2.25,2,True,0
4,CUST0004,M,39,2024-11-04,trimestrale,80,2025-06-06,1.66,16,False,0
